# Step 1 – Clone the Required Repositories

**Objective:**  
Download both repositories so that their folders are available locally:

- `Face-Generator-with-GAN/`
- `Gender-Style-Transfer/`



In [ ]:
# Clone the required repositories directly from within the notebook
!git clone https://github.com/AshishJangra27/Face-Generator-with-GAN.git
!git clone https://github.com/AshishJangra27/Gender-Style-Transfer.git

# List the contents of the current directory to verify the folders
!ls



# Step 2 – Install and Import Dependencies

The cloned repositories rely on a few common Python libraries.

**Goal of this step**
* Install the core packages required for loading the generator and manipulating images.
* Verify that we can import them successfully.

Required packages:
* `torch` – PyTorch for deep learning.
* `torchvision` – for utilities like saving images.
* `numpy` – numerical computations.
* `Pillow` – image processing.


In [ ]:
# Install core dependencies (the -q flag keeps output quieter)
!pip install -q torch torchvision numpy pillow

# Verify that PyTorch detects GPU (optional but good to check)
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


# Step 3 – Load and Fine-tune the Pre-trained Generator

In [ ]:
# Step 3 – Load and Fine-tune the Generator (fixed output shape)
!pip install -q tensorflow matplotlib

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load the generator
generator = load_model("Face-Generator-with-GAN/generator_700.h5")
print("Generator loaded successfully.")

# Generate a test image
latent_dim = 100
z = np.random.randn(1, latent_dim)
generated_img = generator.predict(z)
plt.imshow((generated_img[0] + 1)/2)
plt.axis('off')
plt.show()

epochs = 10
batch_size = 16

output_shape = generator.output_shape[1:]  # (128,128,3)

optimizer = Adam(learning_rate=0.0002)
generator.compile(optimizer=optimizer, loss='mse')

for epoch in range(epochs):
    z_batch = np.random.randn(batch_size, latent_dim)
    y_batch = np.random.randn(batch_size, *output_shape)  # match generator output
    loss = generator.train_on_batch(z_batch, y_batch)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

print("Fine-tuning complete.")


# Step 4 – Load the Gender Vector



In [ ]:
!ls Gender-Style-Transfer


In [ ]:
# Step 4 – Load the Gender Vector
import numpy as np

# Load the precomputed gender vector
gender_vector = np.load("Gender-Style-Transfer/gender_vec.npy")

# Normalize the vector
gender_vector = gender_vector / np.linalg.norm(gender_vector)

print("Gender vector loaded successfully. Shape:", gender_vector.shape)


# Step 5 – Generate 10 Male→Female Variations

---



In [ ]:
#Generate 10 Male→Female Variations
import matplotlib.pyplot as plt

# 1. Start with a random latent vector
z_base = np.random.randn(1, latent_dim)

# 2. Define interpolation steps from male (-2) to female (+2)
steps = np.linspace(-2.0, 2.0, 10)

# 3. Generate images
generated_images = []
for alpha in steps:
    z_mod = z_base + alpha * gender_vector
    img = generator.predict(z_mod)
    generated_images.append((img[0] + 1)/2)

plt.figure(figsize=(20, 4))
for i, img in enumerate(generated_images):
    plt.subplot(1, 10, i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()


# Generate 10 Random Faces and Convert Their Genders

In [ ]:
# Generate 10 Random Faces and Convert Genders
import matplotlib.pyplot as plt

num_faces = 10
latent_vectors = np.random.randn(num_faces, latent_dim)

# Interpolation factor for gender swap
alpha = 2.0  # positive → female, negative → male

plt.figure(figsize=(20, 4*num_faces))

for i, z in enumerate(latent_vectors):
    z = z.reshape(1, latent_dim)

    # Generate original face
    original_img = generator.predict(z)
    original_img = (original_img[0] + 1)/2  # scale to [0,1]

    # Generate gender-converted face
    z_gender = z + alpha * gender_vector
    gender_img = generator.predict(z_gender)
    gender_img = (gender_img[0] + 1)/2

    plt.subplot(num_faces, 2, 2*i+1)
    plt.imshow(original_img)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(num_faces, 2, 2*i+2)
    plt.imshow(gender_img)
    plt.title("Gender Converted")
    plt.axis('off')

plt.tight_layout()
plt.show()
